# LanguageBind + MMS
**CNN**

In [6]:
"""
each branch gets a separate cnn
"""

from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CSV Files
df1 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_lb.csv")  # LB dataset
df2 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_mms.csv")  # MMS dataset

# Extract Labels (assuming both datasets have the same labels)
y = df1["Sarcasm"].values  

# Extract features from both datasets
Xc1 = df1[[col for col in df1.columns if col.startswith("audio_c_feature_")]].values
Xu1 = df1[[col for col in df1.columns if col.startswith("audio_u_feature_")]].values
Xc2 = df2[[col for col in df2.columns if col.startswith("audio_c_feature_")]].values
Xu2 = df2[[col for col in df2.columns if col.startswith("audio_u_feature_")]].values

# Convert to NumPy arrays
Xc1, Xu1, Xc2, Xu2 = map(lambda x: np.array(x, dtype=np.float32), [Xc1, Xu1, Xc2, Xu2])
y = np.array(y, dtype=np.float32)

# Train-test split (70%-30%)
Xc1_train, Xc1_temp, Xu1_train, Xu1_temp, Xc2_train, Xc2_temp, Xu2_train, Xu2_temp, y_train, y_temp = train_test_split(
    Xc1, Xu1, Xc2, Xu2, y, test_size=0.3, random_state=42, stratify=y
)

# Further split temp set into validation (10%) and test (20%)
Xc1_val, Xc1_test, Xu1_val, Xu1_test, Xc2_val, Xc2_test, Xu2_val, Xu2_test, y_val, y_test = train_test_split(
    Xc1_temp, Xu1_temp, Xc2_temp, Xu2_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

# Define feature dimensions separately
input_dim_lb = 768  # LB Feature dimension
input_dim_mms = 1280  # MMS Feature dimension

# CNN Model for feature extraction
def build_cnn_branch(input_dim):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Reshape((input_dim, 1))(inp)
    x = layers.Conv1D(filters=126, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    return inp, x

# Apply CNN on LB inputs (768)
input_c1, context_cnn1 = build_cnn_branch(input_dim_lb)
input_u1, utterance_cnn1 = build_cnn_branch(input_dim_lb)

# Apply CNN on MMS inputs (1280)
input_c2, context_cnn2 = build_cnn_branch(input_dim_mms)
input_u2, utterance_cnn2 = build_cnn_branch(input_dim_mms)

# First fusion (context_cnn1 + utterance_cnn1) and (context_cnn2 + utterance_cnn2)
fused_1 = layers.Concatenate()([context_cnn1, utterance_cnn1])
fused_2 = layers.Concatenate()([context_cnn2, utterance_cnn2])

# Apply another CNN on fused representations
def build_fused_cnn(input_tensor):
    x = layers.Reshape((-1, 1))(input_tensor) 
    x = layers.Conv1D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=32, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    return x

def build_fused_fcn(input_tensor):
    x = layers.Dense(128, activation="swish")(input_tensor)
    x = layers.Dense(64, activation="swish")(x)
    x = layers.Dense(32, activation="swish")(x)
    return x

cnn1 = build_fused_fcn(fused_1)
cnn2 = build_fused_fcn(fused_2)

# Final fusion (cnn1 + cnn2)
final_fusion = layers.Concatenate()([cnn1, cnn2])

# Fully connected layers
fc = layers.Dense(64, activation="swish")(final_fusion)
fc = layers.Dense(32, activation="swish")(fc)
output = layers.Dense(1, activation="sigmoid")(fc)

# Define and compile model
model = keras.Model(inputs=[input_c1, input_u1, input_c2, input_u2], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Model checkpoint
checkpoint_callback = ModelCheckpoint(
    "/kaggle/working/lb+mms_cnn.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Train Model
model.fit(
    [Xc1_train, Xu1_train, Xc2_train, Xu2_train], y_train,
    epochs=50, batch_size=32,
    validation_data=([Xc1_val, Xu1_val, Xc2_val, Xu2_val], y_val),
    callbacks=[checkpoint_callback]
)

# Load best weights
model.load_weights("/kaggle/working/lb+mms_cnn.weights.h5")
print("Loaded Best Model Weights.")

# Predictions
y_train_pred = (model.predict([Xc1_train, Xu1_train, Xc2_train, Xu2_train]) > 0.5).astype(int)
y_val_pred = (model.predict([Xc1_val, Xu1_val, Xc2_val, Xu2_val]) > 0.5).astype(int)
y_test_pred = (model.predict([Xc1_test, Xu1_test, Xc2_test, Xu2_test]) > 0.5).astype(int)

# Classification Reports
print("Train Set Classification Report:\n", classification_report(y_train, y_train_pred, digits=4))
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14            │ (None, 768)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_15            │ (None, 768)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_16            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_17            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_16 (Reshape)      │ (None, 768, 1)         │              0 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_17 (Reshape)      │ (None, 768, 1)         │              0 │ input_layer_15[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_18 (Reshape)      │ (None, 1280, 1)        │              0 │ input_layer_16[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_19 (Reshape)      │ (None, 1280, 1)        │              0 │ input_layer_17[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_32 (Conv1D)        │ (None, 766, 126)       │            504 │ reshape_16[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_34 (Conv1D)        │ (None, 766, 126)       │            504 │ reshape_17[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_36 (Conv1D)        │ (None, 1278, 126)      │            504 │ reshape_18[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_38 (Conv1D)        │ (None, 1278, 126)      │            504 │ reshape_19[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_32          │ (None, 383, 126)       │              0 │ conv1d_32[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_34          │ (None, 383, 126)       │              0 │ conv1d_34[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_36          │ (None, 639, 126)       │              0 │ conv1d_36[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_38          │ (None, 639, 126)       │              0 │ conv1d_38[0][0]        │
│ (MaxPooling1D)       

 Total params: 8,449,313 (32.23 MB)

 Trainable params: 8,449,313 (32.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.4927 - loss: 0.7026
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to /kaggle/working/lb+mms_cnn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 160ms/step - accuracy: 0.4936 - loss: 0.7024 - val_accuracy: 0.5000 - val_loss: 0.6907
Epoch 2/50
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5525 - loss: 0.6836
Epoch 2: val_accuracy improved from 0.50000 to 0.57500, saving model to /kaggle/working/lb+mms_cnn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5554 - loss: 0.6829 - val_accuracy: 0.5750 - val_loss: 0.6566
Epoch 3/50
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6350 - loss: 0.6529
Epoch 3: val_accuracy improved from 0.57500 to 0.67500, saving model to /kaggle/working/lb+mms_cnn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.6338 - loss: 0.6519 - val_accuracy: 0.6750 - val_loss: 0.6313
Epoch 4/50
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.66

**FCN**

In [1]:
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CSV Files
df1 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_lb.csv")  # LB dataset
df2 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_mms.csv")  # MMS dataset

# Extract Labels
y = df1["Sarcasm"].values  

# Extract features from both datasets
Xc1 = df1[[col for col in df1.columns if col.startswith("audio_c_feature_")]].values
Xu1 = df1[[col for col in df1.columns if col.startswith("audio_u_feature_")]].values
Xc2 = df2[[col for col in df2.columns if col.startswith("audio_c_feature_")]].values
Xu2 = df2[[col for col in df2.columns if col.startswith("audio_u_feature_")]].values

# Convert to NumPy arrays
Xc1, Xu1, Xc2, Xu2 = map(lambda x: np.array(x, dtype=np.float32), [Xc1, Xu1, Xc2, Xu2])
y = np.array(y, dtype=np.float32)

# Train-test split
Xc1_train, Xc1_temp, Xu1_train, Xu1_temp, Xc2_train, Xc2_temp, Xu2_train, Xu2_temp, y_train, y_temp = train_test_split(
    Xc1, Xu1, Xc2, Xu2, y, test_size=0.3, random_state=42, stratify=y
)

Xc1_val, Xc1_test, Xu1_val, Xu1_test, Xc2_val, Xc2_test, Xu2_val, Xu2_test, y_val, y_test = train_test_split(
    Xc1_temp, Xu1_temp, Xc2_temp, Xu2_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

# Define feature dimensions
input_dim_lb = 768  # LB Feature dimension
input_dim_mms = 1280  # MMS Feature dimension

# Fully connected network branch
def build_fcn_branch(input_dim):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Dense(512, activation="swish")(inp)
    x = layers.Dense(256, activation="swish")(x)
    x = layers.Dense(128, activation="swish")(x)
    return inp, x

# FCN for LB dataset
input_c1, context_fcn1 = build_fcn_branch(input_dim_lb)
input_u1, utterance_fcn1 = build_fcn_branch(input_dim_lb)

# FCN for MMS dataset
input_c2, context_fcn2 = build_fcn_branch(input_dim_mms)
input_u2, utterance_fcn2 = build_fcn_branch(input_dim_mms)

# First fusion
fused_1 = layers.Concatenate()([context_fcn1, utterance_fcn1])
fused_2 = layers.Concatenate()([context_fcn2, utterance_fcn2])

# Additional FCN model after fusion
def build_fcn_model(input_tensor):
    x = layers.Dense(512, activation="swish")(input_tensor)
    x = layers.Dense(256, activation="swish")(x)
    x = layers.Dense(128, activation="swish")(x)
    return x

fcn1 = build_fcn_model(fused_1)
fcn2 = build_fcn_model(fused_2)

# Final fusion
final_fusion = layers.Concatenate()([fcn1, fcn2])

# Fully connected layers
fc = layers.Dense(64, activation="swish")(final_fusion)
fc = layers.Dense(32, activation="swish")(fc)
fc = layers.Dense(8, activation="swish")(fc)
output = layers.Dense(1, activation="sigmoid")(fc)

# Define and compile model
model = keras.Model(inputs=[input_c1, input_u1, input_c2, input_u2], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Model checkpoint
checkpoint_callback = ModelCheckpoint(
    "/kaggle/working/lb+mms_fcn.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Train Model
model.fit(
    [Xc1_train, Xu1_train, Xc2_train, Xu2_train], y_train,
    epochs=50, batch_size=32,
    validation_data=([Xc1_val, Xu1_val, Xc2_val, Xu2_val], y_val),
    callbacks=[checkpoint_callback]
)

# Load best weights
model.load_weights("/kaggle/working/lb+mms_fcn.weights.h5")
print("Loaded Best Model Weights.")

# Predictions
y_train_pred = (model.predict([Xc1_train, Xu1_train, Xc2_train, Xu2_train]) > 0.5).astype(int)
y_val_pred = (model.predict([Xc1_val, Xu1_val, Xc2_val, Xu2_val]) > 0.5).astype(int)
y_test_pred = (model.predict([Xc1_test, Xu1_test, Xc2_test, Xu2_test]) > 0.5).astype(int)

# Classification Reports
print("Train Set Classification Report:\n", classification_report(y_train, y_train_pred, digits=4))
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 768)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 768)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 512)            │        393,728 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 512)            │        393,728 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 512)            │        655,872 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 512)            │        655,872 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 256)            │        131,328 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 256)            │        131,328 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 256)            │        131,328 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 256)            │        131,328 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │         32,896 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 128)            │         32,896 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 128)            │         32,896 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 128)            │         32,896 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 256)            │              0 │ dense_2[0][0],         │
│                           │                        │                │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1        

 Total params: 3,366,513 (12.84 MB)

 Trainable params: 3,366,513 (12.84 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.4890 - loss: 0.6936
Epoch 1: val_accuracy improved from -inf to 0.60000, saving model to /kaggle/working/lb+mms_fcn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 146ms/step - accuracy: 0.4907 - loss: 0.6933 - val_accuracy: 0.6000 - val_loss: 0.6500
Epoch 2/50
16/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6748 - loss: 0.6212 
Epoch 2: val_accuracy improved from 0.60000 to 0.66667, saving model to /kaggle/working/lb+mms_fcn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6693 - loss: 0.6244 - val_accuracy: 0.6667 - val_loss: 0.6139
Epoch 3/50
16/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6427 - loss: 0.6143 
Epoch 3: val_accuracy improved from 0.66667 to 0.68333, saving model to /kaggle/working/lb+mms_fcn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6604 - loss: 0.6126 - val_accuracy: 0.6833 - val_loss: 0.6083
Epoch 4/50
17/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6667

# LanguageBind + XLs-R
**CNN**

In [3]:
"""
each branch gets a separate cnn
"""

from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CSV Files
df1 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_lb.csv")  # LB dataset
df2 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_xlsr.csv")  # MMS dataset

# Extract Labels (assuming both datasets have the same labels)
y = df1["Sarcasm"].values  

# Extract features from both datasets
Xc1 = df1[[col for col in df1.columns if col.startswith("audio_c_feature_")]].values
Xu1 = df1[[col for col in df1.columns if col.startswith("audio_u_feature_")]].values
Xc2 = df2[[col for col in df2.columns if col.startswith("audio_c_feature_")]].values
Xu2 = df2[[col for col in df2.columns if col.startswith("audio_u_feature_")]].values

# Convert to NumPy arrays
Xc1, Xu1, Xc2, Xu2 = map(lambda x: np.array(x, dtype=np.float32), [Xc1, Xu1, Xc2, Xu2])
y = np.array(y, dtype=np.float32)

# Train-test split (70%-30%)
Xc1_train, Xc1_temp, Xu1_train, Xu1_temp, Xc2_train, Xc2_temp, Xu2_train, Xu2_temp, y_train, y_temp = train_test_split(
    Xc1, Xu1, Xc2, Xu2, y, test_size=0.3, random_state=42, stratify=y
)

# Further split temp set into validation (10%) and test (20%)
Xc1_val, Xc1_test, Xu1_val, Xu1_test, Xc2_val, Xc2_test, Xu2_val, Xu2_test, y_val, y_test = train_test_split(
    Xc1_temp, Xu1_temp, Xc2_temp, Xu2_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

# Define feature dimensions separately
input_dim_lb = 768  # LB Feature dimension
input_dim_mms = 1280  # MMS Feature dimension

# CNN Model for feature extraction
def build_cnn_branch(input_dim):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Reshape((input_dim, 1))(inp)
    x = layers.Conv1D(filters=126, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    return inp, x

# Apply CNN on LB inputs (768)
input_c1, context_cnn1 = build_cnn_branch(input_dim_lb)
input_u1, utterance_cnn1 = build_cnn_branch(input_dim_lb)

# Apply CNN on MMS inputs (1280)
input_c2, context_cnn2 = build_cnn_branch(input_dim_mms)
input_u2, utterance_cnn2 = build_cnn_branch(input_dim_mms)

# First fusion (context_cnn1 + utterance_cnn1) and (context_cnn2 + utterance_cnn2)
fused_1 = layers.Concatenate()([context_cnn1, utterance_cnn1])
fused_2 = layers.Concatenate()([context_cnn2, utterance_cnn2])


def build_fused_fcn(input_tensor):
    x = layers.Dense(128, activation="relu")(input_tensor)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    return x

cnn1 = build_fused_fcn(fused_1)
cnn2 = build_fused_fcn(fused_2)

# Final fusion (cnn1 + cnn2)
final_fusion = layers.Concatenate()([cnn1, cnn2])

# Fully connected layers
fc = layers.Dense(64, activation="relu")(final_fusion)
fc = layers.Dense(32, activation="relu")(fc)
output = layers.Dense(1, activation="sigmoid")(fc)

# Define and compile model
model = keras.Model(inputs=[input_c1, input_u1, input_c2, input_u2], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Model checkpoint
checkpoint_callback = ModelCheckpoint(
    "/kaggle/working/lb+xlsr_cnn.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Train Model
model.fit(
    [Xc1_train, Xu1_train, Xc2_train, Xu2_train], y_train,
    epochs=50, batch_size=32,
    validation_data=([Xc1_val, Xu1_val, Xc2_val, Xu2_val], y_val),
    callbacks=[checkpoint_callback]
)

# Load best weights
model.load_weights("/kaggle/working/lb+xlsr_cnn.weights.h5")
print("Loaded Best Model Weights.")

# Predictions
y_train_pred = (model.predict([Xc1_train, Xu1_train, Xc2_train, Xu2_train]) > 0.5).astype(int)
y_val_pred = (model.predict([Xc1_val, Xu1_val, Xc2_val, Xu2_val]) > 0.5).astype(int)
y_test_pred = (model.predict([Xc1_test, Xu1_test, Xc2_test, Xu2_test]) > 0.5).astype(int)

# Classification Reports
print("Train Set Classification Report:\n", classification_report(y_train, y_train_pred, digits=4))
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 768)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_9             │ (None, 768)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_10            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_11            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_4 (Reshape)       │ (None, 768, 1)         │              0 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_5 (Reshape)       │ (None, 768, 1)         │              0 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_6 (Reshape)       │ (None, 1280, 1)        │              0 │ input_layer_10[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_7 (Reshape)       │ (None, 1280, 1)        │              0 │ input_layer_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_8 (Conv1D)         │ (None, 766, 126)       │            504 │ reshape_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_10 (Conv1D)        │ (None, 766, 126)       │            504 │ reshape_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_12 (Conv1D)        │ (None, 1278, 126)      │            504 │ reshape_6[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_14 (Conv1D)        │ (None, 1278, 126)      │            504 │ reshape_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_8           │ (None, 383, 126)       │              0 │ conv1d_8[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_10          │ (None, 383, 126)       │              0 │ conv1d_10[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_12          │ (None, 639, 126)       │              0 │ conv1d_12[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_14          │ (None, 639, 126)       │              0 │ conv1d_14[0][0]        │
│ (MaxPooling1D)       

 Total params: 8,449,313 (32.23 MB)

 Trainable params: 8,449,313 (32.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.5481 - loss: 0.6927
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to /kaggle/working/lb+xlsr_cnn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 162ms/step - accuracy: 0.5483 - loss: 0.6926 - val_accuracy: 0.5000 - val_loss: 0.7024
Epoch 2/50
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5521 - loss: 0.6804
Epoch 2: val_accuracy improved from 0.50000 to 0.60833, saving model to /kaggle/working/lb+xlsr_cnn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.5566 - loss: 0.6795 - val_accuracy: 0.6083 - val_loss: 0.6533
Epoch 3/50
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6141 - loss: 0.6469
Epoch 3: val_accuracy did not improve from 0.60833
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6124 - loss: 0.6485 - val_accuracy: 0.5000 - val_loss: 0.6769
Epoch 4/50
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5854 - loss: 0.6666
Epoch 4: val_accuracy improved from 0.60

**FCN**

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CSV Files
df1 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_lb.csv")  # LB dataset
df2 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_xlsr.csv")  # MMS dataset

# Extract Labels
y = df1["Sarcasm"].values  

# Extract features from both datasets
Xc1 = df1[[col for col in df1.columns if col.startswith("audio_c_feature_")]].values
Xu1 = df1[[col for col in df1.columns if col.startswith("audio_u_feature_")]].values
Xc2 = df2[[col for col in df2.columns if col.startswith("audio_c_feature_")]].values
Xu2 = df2[[col for col in df2.columns if col.startswith("audio_u_feature_")]].values

# Convert to NumPy arrays
Xc1, Xu1, Xc2, Xu2 = map(lambda x: np.array(x, dtype=np.float32), [Xc1, Xu1, Xc2, Xu2])
y = np.array(y, dtype=np.float32)

# Train-test split
Xc1_train, Xc1_temp, Xu1_train, Xu1_temp, Xc2_train, Xc2_temp, Xu2_train, Xu2_temp, y_train, y_temp = train_test_split(
    Xc1, Xu1, Xc2, Xu2, y, test_size=0.3, random_state=42, stratify=y
)

Xc1_val, Xc1_test, Xu1_val, Xu1_test, Xc2_val, Xc2_test, Xu2_val, Xu2_test, y_val, y_test = train_test_split(
    Xc1_temp, Xu1_temp, Xc2_temp, Xu2_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

# Define feature dimensions
input_dim_lb = 768  # LB Feature dimension
input_dim_mms = 1280  # MMS Feature dimension

# Fully connected network branch
def build_fcn_branch(input_dim):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Dense(512, activation="relu")(inp)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(x)
    return inp, x

# FCN for LB dataset
input_c1, context_fcn1 = build_fcn_branch(input_dim_lb)
input_u1, utterance_fcn1 = build_fcn_branch(input_dim_lb)

# FCN for MMS dataset
input_c2, context_fcn2 = build_fcn_branch(input_dim_mms)
input_u2, utterance_fcn2 = build_fcn_branch(input_dim_mms)

# First fusion
fused_1 = layers.Concatenate()([context_fcn1, utterance_fcn1])
fused_2 = layers.Concatenate()([context_fcn2, utterance_fcn2])

# Additional FCN model after fusion
def build_fcn_model(input_tensor):
    x = layers.Dense(512, activation="relu")(input_tensor)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(x)
    return x

fcn1 = build_fcn_model(fused_1)
fcn2 = build_fcn_model(fused_2)

# Final fusion
final_fusion = layers.Concatenate()([fcn1, fcn2])

# Fully connected layers
fc = layers.Dense(64, activation="relu")(final_fusion)
fc = layers.Dense(32, activation="relu")(fc)
fc = layers.Dense(8, activation="relu")(fc)
output = layers.Dense(1, activation="sigmoid")(fc)

# Define and compile model
model = keras.Model(inputs=[input_c1, input_u1, input_c2, input_u2], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Model checkpoint
checkpoint_callback = ModelCheckpoint(
    "/kaggle/working/lb+xlsr_fcn.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Train Model
model.fit(
    [Xc1_train, Xu1_train, Xc2_train, Xu2_train], y_train,
    epochs=50, batch_size=32,
    validation_data=([Xc1_val, Xu1_val, Xc2_val, Xu2_val], y_val),
    callbacks=[checkpoint_callback]
)

# Load best weights
model.load_weights("/kaggle/working/lb+xlsr_fcn.weights.h5")
print("Loaded Best Model Weights.")

# Predictions
y_train_pred = (model.predict([Xc1_train, Xu1_train, Xc2_train, Xu2_train]) > 0.5).astype(int)
y_val_pred = (model.predict([Xc1_val, Xu1_val, Xc2_val, Xu2_val]) > 0.5).astype(int)
y_test_pred = (model.predict([Xc1_test, Xu1_test, Xc2_test, Xu2_test]) > 0.5).astype(int)

# Classification Reports
print("Train Set Classification Report:\n", classification_report(y_train, y_train_pred, digits=4))
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20            │ (None, 768)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_21            │ (None, 768)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_22            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_23            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_84 (Dense)          │ (None, 512)            │        393,728 │ input_layer_20[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_87 (Dense)          │ (None, 512)            │        393,728 │ input_layer_21[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_90 (Dense)          │ (None, 512)            │        655,872 │ input_layer_22[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_93 (Dense)          │ (None, 512)            │        655,872 │ input_layer_23[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_85 (Dense)          │ (None, 256)            │        131,328 │ dense_84[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_88 (Dense)          │ (None, 256)            │        131,328 │ dense_87[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_91 (Dense)          │ (None, 256)            │        131,328 │ dense_90[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_94 (Dense)          │ (None, 256)            │        131,328 │ dense_93[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_86 (Dense)          │ (None, 128)            │         32,896 │ dense_85[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_89 (Dense)          │ (None, 128)            │         32,896 │ dense_88[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_92 (Dense)          │ (None, 128)            │         32,896 │ dense_91[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_95 (Dense)          │ (None, 128)            │         32,896 │ dense_94[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_15            │ (None, 256)            │              0 │ dense_86[0][0],        │
│ (Concatenate)             │                        │                │ dense_89[0][0]         │
├──────────────────────

 Total params: 3,366,513 (12.84 MB)

 Trainable params: 3,366,513 (12.84 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5215 - loss: 0.6940
Epoch 1: val_accuracy improved from -inf to 0.64167, saving model to /kaggle/working/lb+xlsr_fcn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - accuracy: 0.5225 - loss: 0.6936 - val_accuracy: 0.6417 - val_loss: 0.6311
Epoch 2/50
16/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6880 - loss: 0.6009 
Epoch 2: val_accuracy improved from 0.64167 to 0.70000, saving model to /kaggle/working/lb+xlsr_fcn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6755 - loss: 0.6137 - val_accuracy: 0.7000 - val_loss: 0.6147
Epoch 3/50
16/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6826 - loss: 0.5796 
Epoch 3: val_accuracy did not improve from 0.70000
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6878 - loss: 0.5805 - val_accuracy: 0.6917 - val_loss: 0.6205
Epoch 4/50
16/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7619 - loss: 0.4999 
Epoch 4: val_accuracy did not improve from 

# Whisper + MMS
**CNN**

In [19]:
"""
each branch gets a separate cnn
"""

from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CSV Files
df1 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_whisper.csv")  # LB dataset
df2 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_mms.csv")  # MMS dataset

# Extract Labels (assuming both datasets have the same labels)
y = df1["Sarcasm"].values  

# Extract features from both datasets
Xc1 = df1[[col for col in df1.columns if col.startswith("audio_c_feature_")]].values
Xu1 = df1[[col for col in df1.columns if col.startswith("audio_u_feature_")]].values
Xc2 = df2[[col for col in df2.columns if col.startswith("audio_c_feature_")]].values
Xu2 = df2[[col for col in df2.columns if col.startswith("audio_u_feature_")]].values

# Convert to NumPy arrays
Xc1, Xu1, Xc2, Xu2 = map(lambda x: np.array(x, dtype=np.float32), [Xc1, Xu1, Xc2, Xu2])
y = np.array(y, dtype=np.float32)

# Train-test split (70%-30%)
Xc1_train, Xc1_temp, Xu1_train, Xu1_temp, Xc2_train, Xc2_temp, Xu2_train, Xu2_temp, y_train, y_temp = train_test_split(
    Xc1, Xu1, Xc2, Xu2, y, test_size=0.3, random_state=42, stratify=y
)

# Further split temp set into validation (10%) and test (20%)
Xc1_val, Xc1_test, Xu1_val, Xu1_test, Xc2_val, Xc2_test, Xu2_val, Xu2_test, y_val, y_test = train_test_split(
    Xc1_temp, Xu1_temp, Xc2_temp, Xu2_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

# Define feature dimensions separately
input_dim_lb = 512  # LB Feature dimension
input_dim_mms = 1280  # MMS Feature dimension

# CNN Model for feature extraction
def build_cnn_branch(input_dim):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Reshape((input_dim, 1))(inp)
    x = layers.Conv1D(filters=264, kernel_size=3, activation="swish")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=126, kernel_size=3, activation="swish")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=64, kernel_size=3, activation="swish")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    return inp, x

# Apply CNN on LB inputs (768)
input_c1, context_cnn1 = build_cnn_branch(input_dim_lb)
input_u1, utterance_cnn1 = build_cnn_branch(input_dim_lb)

# Apply CNN on MMS inputs (1280)
input_c2, context_cnn2 = build_cnn_branch(input_dim_mms)
input_u2, utterance_cnn2 = build_cnn_branch(input_dim_mms)

# First fusion (context_cnn1 + utterance_cnn1) and (context_cnn2 + utterance_cnn2)
fused_1 = layers.Concatenate()([context_cnn1, utterance_cnn1])
fused_2 = layers.Concatenate()([context_cnn2, utterance_cnn2])



def build_fused_fcn(input_tensor):
    x = layers.Dense(264, activation="swish")(input_tensor)
    x = layers.Dense(126, activation="swish")(x)
    x = layers.Dense(64, activation="swish")(x)
    x = layers.Dense(32, activation="swish")(x)
    x = layers.Dense(8, activation="swish")(x)
    return x

cnn1 = build_fused_fcn(fused_1)
cnn2 = build_fused_fcn(fused_2)

# Final fusion (cnn1 + cnn2)
final_fusion = layers.Concatenate()([cnn1, cnn2])

# Fully connected layers
fc = layers.Dense(64, activation="swish")(final_fusion)
fc = layers.Dense(32, activation="swish")(fc)
fc = layers.Dense(8, activation="swish")(fc)
output = layers.Dense(1, activation="sigmoid")(fc)

# Define and compile model
model = keras.Model(inputs=[input_c1, input_u1, input_c2, input_u2], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Model checkpoint
checkpoint_callback = ModelCheckpoint(
    "/kaggle/working/whisper+mms_cnn.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Train Model
model.fit(
    [Xc1_train, Xu1_train, Xc2_train, Xu2_train], y_train,
    epochs=50, batch_size=32,
    validation_data=([Xc1_val, Xu1_val, Xc2_val, Xu2_val], y_val),
    callbacks=[checkpoint_callback]
)

# Load best weights
model.load_weights("/kaggle/working/whisper+mms_cnn.weights.h5")
print("Loaded Best Model Weights.")

# Predictions
y_train_pred = (model.predict([Xc1_train, Xu1_train, Xc2_train, Xu2_train]) > 0.5).astype(int)
y_val_pred = (model.predict([Xc1_val, Xu1_val, Xc2_val, Xu2_val]) > 0.5).astype(int)
y_test_pred = (model.predict([Xc1_test, Xu1_test, Xc2_test, Xu2_test]) > 0.5).astype(int)

# Classification Reports
print("Train Set Classification Report:\n", classification_report(y_train, y_train_pred, digits=4))
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_72            │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_73            │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_74            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_75            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_56 (Reshape)      │ (None, 512, 1)         │              0 │ input_layer_72[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_57 (Reshape)      │ (None, 512, 1)         │              0 │ input_layer_73[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_58 (Reshape)      │ (None, 1280, 1)        │              0 │ input_layer_74[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_59 (Reshape)      │ (None, 1280, 1)        │              0 │ input_layer_75[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_140 (Conv1D)       │ (None, 510, 264)       │          1,056 │ reshape_56[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_143 (Conv1D)       │ (None, 510, 264)       │          1,056 │ reshape_57[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_146 (Conv1D)       │ (None, 1278, 264)      │          1,056 │ reshape_58[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_149 (Conv1D)       │ (None, 1278, 264)      │          1,056 │ reshape_59[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_140         │ (None, 255, 264)       │              0 │ conv1d_140[0][0]       │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_143         │ (None, 255, 264)       │              0 │ conv1d_143[0][0]       │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_146         │ (None, 639, 264)       │              0 │ conv1d_146[0][0]       │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_149         │ (None, 639, 264)       │              0 │ conv1d_149[0][0]       │
│ (MaxPooling1D)       

 Total params: 8,026,853 (30.62 MB)

 Trainable params: 8,026,853 (30.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.5049 - loss: 0.6936
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to /kaggle/working/whisper+mms_cnn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 183ms/step - accuracy: 0.5045 - loss: 0.6936 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/50
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4530 - loss: 0.6937
Epoch 2: val_accuracy did not improve from 0.50000
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4528 - loss: 0.6938 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/50
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5017 - loss: 0.6931
Epoch 3: val_accuracy did not improve from 0.50000
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.5013 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/50
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4579 - loss: 0.6934
Epoch 4: val_accuracy improved from 0.50000 to 0.53333, saving model to /kaggle/working/whisper

**FCN**

In [30]:
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CSV Files
df1 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_whisper.csv")  # LB dataset
df2 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_mms.csv")  # MMS dataset

# Extract Labels
y = df1["Sarcasm"].values  

# Extract features from both datasets
Xc1 = df1[[col for col in df1.columns if col.startswith("audio_c_feature_")]].values
Xu1 = df1[[col for col in df1.columns if col.startswith("audio_u_feature_")]].values
Xc2 = df2[[col for col in df2.columns if col.startswith("audio_c_feature_")]].values
Xu2 = df2[[col for col in df2.columns if col.startswith("audio_u_feature_")]].values

# Convert to NumPy arrays
Xc1, Xu1, Xc2, Xu2 = map(lambda x: np.array(x, dtype=np.float32), [Xc1, Xu1, Xc2, Xu2])
y = np.array(y, dtype=np.float32)

# Train-test split
Xc1_train, Xc1_temp, Xu1_train, Xu1_temp, Xc2_train, Xc2_temp, Xu2_train, Xu2_temp, y_train, y_temp = train_test_split(
    Xc1, Xu1, Xc2, Xu2, y, test_size=0.3, random_state=42, stratify=y
)

Xc1_val, Xc1_test, Xu1_val, Xu1_test, Xc2_val, Xc2_test, Xu2_val, Xu2_test, y_val, y_test = train_test_split(
    Xc1_temp, Xu1_temp, Xc2_temp, Xu2_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

# Define feature dimensions
input_dim_lb = 512  # LB Feature dimension
input_dim_mms = 1280  # MMS Feature dimension

# Fully connected network branch
def build_fcn_branch(input_dim):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Dense(684, activation="swish")(inp)
    x = layers.Dense(512, activation="swish")(x)
    x = layers.Dense(256, activation="swish")(x)
    x = layers.Dense(128, activation="swish")(x)
    x = layers.Dense(64, activation="swish")(x)
    x = layers.Dense(32, activation="swish")(x)
    x = layers.Dense(8, activation="swish")(x)
    return inp, x

# FCN for LB dataset
input_c1, context_fcn1 = build_fcn_branch(input_dim_lb)
input_u1, utterance_fcn1 = build_fcn_branch(input_dim_lb)

# FCN for MMS dataset
input_c2, context_fcn2 = build_fcn_branch(input_dim_mms)
input_u2, utterance_fcn2 = build_fcn_branch(input_dim_mms)

# First fusion
fused_1 = layers.Concatenate()([context_fcn1, utterance_fcn1])
fused_2 = layers.Concatenate()([context_fcn2, utterance_fcn2])

# Additional FCN model after fusion
def build_fcn_model(input_tensor):
    x = layers.Dense(512, activation="swish")(input_tensor)
    x = layers.Dense(256, activation="swish")(x)
    x = layers.Dense(128, activation="swish")(x)
    x = layers.Dense(64, activation="swish")(x)
    x = layers.Dense(32, activation="swish")(x)
    return x

fcn1 = build_fcn_model(fused_1)
fcn2 = build_fcn_model(fused_2)

# Final fusion
final_fusion = layers.Concatenate()([fcn1, fcn2])

# Fully connected layers
fc = layers.Dense(64, activation="swish")(final_fusion)
fc = layers.Dense(32, activation="swish")(fc)
fc = layers.Dense(8, activation="swish")(fc)
output = layers.Dense(1, activation="sigmoid")(fc)

# Define and compile model
model = keras.Model(inputs=[input_c1, input_u1, input_c2, input_u2], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Model checkpoint
checkpoint_callback = ModelCheckpoint(
    "/kaggle/working/whisper+mms_fcn.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Train Model
model.fit(
    [Xc1_train, Xu1_train, Xc2_train, Xu2_train], y_train,
    epochs=50, batch_size=32,
    validation_data=([Xc1_val, Xu1_val, Xc2_val, Xu2_val], y_val),
    callbacks=[checkpoint_callback]
)

# Load best weights
model.load_weights("/kaggle/working/whisper+mms_fcn.weights.h5")
print("Loaded Best Model Weights.")

# Predictions
y_train_pred = (model.predict([Xc1_train, Xu1_train, Xc2_train, Xu2_train]) > 0.5).astype(int)
y_val_pred = (model.predict([Xc1_val, Xu1_val, Xc2_val, Xu2_val]) > 0.5).astype(int)
y_test_pred = (model.predict([Xc1_test, Xu1_test, Xc2_test, Xu2_test]) > 0.5).astype(int)

# Classification Reports
print("Train Set Classification Report:\n", classification_report(y_train, y_train_pred, digits=4))
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


Model: "functional_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_116           │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_117           │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_118           │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_119           │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_522 (Dense)         │ (None, 684)            │        350,892 │ input_layer_116[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_529 (Dense)         │ (None, 684)            │        350,892 │ input_layer_117[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_536 (Dense)         │ (None, 684)            │        876,204 │ input_layer_118[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_543 (Dense)         │ (None, 684)            │        876,204 │ input_layer_119[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_523 (Dense)         │ (None, 512)            │        350,720 │ dense_522[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_530 (Dense)         │ (None, 512)            │        350,720 │ dense_529[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_537 (Dense)         │ (None, 512)            │        350,720 │ dense_536[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_544 (Dense)         │ (None, 512)            │        350,720 │ dense_543[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_524 (Dense)         │ (None, 256)            │        131,328 │ dense_523[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_531 (Dense)         │ (None, 256)            │        131,328 │ dense_530[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_538 (Dense)         │ (None, 256)            │        131,328 │ dense_537[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_545 (Dense)         │ (None, 256)            │        131,328 │ dense_544[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_525 (Dense)         │ (None, 128)            │         32,896 │ dense_524[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_532 (Dense)    

 Total params: 4,929,409 (18.80 MB)

 Trainable params: 4,929,409 (18.80 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.5033 - loss: 0.6934
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to /kaggle/working/whisper+mms_fcn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 206ms/step - accuracy: 0.5031 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/50
22/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5190 - loss: 0.6930
Epoch 2: val_accuracy did not improve from 0.50000
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5222 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/50
23/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4904 - loss: 0.6933
Epoch 3: val_accuracy did not improve from 0.50000
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4903 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/50
20/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5236 - loss: 0.6932
Epoch 4: val_accuracy did not improve from 0.50000
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy

# Whisper + XLS-R
**CNN**

In [4]:
"""
each branch gets a separate cnn
"""

from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CSV Files
df1 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_whisper.csv")  # LB dataset
df2 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_xlsr.csv")  # MMS dataset

# Extract Labels (assuming both datasets have the same labels)
y = df1["Sarcasm"].values  

# Extract features from both datasets
Xc1 = df1[[col for col in df1.columns if col.startswith("audio_c_feature_")]].values
Xu1 = df1[[col for col in df1.columns if col.startswith("audio_u_feature_")]].values
Xc2 = df2[[col for col in df2.columns if col.startswith("audio_c_feature_")]].values
Xu2 = df2[[col for col in df2.columns if col.startswith("audio_u_feature_")]].values

# Convert to NumPy arrays
Xc1, Xu1, Xc2, Xu2 = map(lambda x: np.array(x, dtype=np.float32), [Xc1, Xu1, Xc2, Xu2])
y = np.array(y, dtype=np.float32)

# Train-test split (70%-30%)
Xc1_train, Xc1_temp, Xu1_train, Xu1_temp, Xc2_train, Xc2_temp, Xu2_train, Xu2_temp, y_train, y_temp = train_test_split(
    Xc1, Xu1, Xc2, Xu2, y, test_size=0.3, random_state=42, stratify=y
)

# Further split temp set into validation (10%) and test (20%)
Xc1_val, Xc1_test, Xu1_val, Xu1_test, Xc2_val, Xc2_test, Xu2_val, Xu2_test, y_val, y_test = train_test_split(
    Xc1_temp, Xu1_temp, Xc2_temp, Xu2_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

# Define feature dimensions separately
input_dim_lb = 512  # LB Feature dimension
input_dim_mms = 1280  # MMS Feature dimension

# CNN Model for feature extraction
def build_cnn_branch(input_dim):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Reshape((input_dim, 1))(inp)
    #x = layers.Conv1D(filters=1280, kernel_size=3, activation="swish")(x)
    #x = layers.MaxPooling1D(pool_size=2)(x)
    #x = layers.Conv1D(filters=512, kernel_size=3, activation="swish")(x)
    #x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=256, kernel_size=3, activation="swish")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=64, kernel_size=3, activation="swish")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    return inp, x

# Apply CNN on LB inputs (768)
input_c1, context_cnn1 = build_cnn_branch(input_dim_lb)
input_u1, utterance_cnn1 = build_cnn_branch(input_dim_lb)

# Apply CNN on MMS inputs (1280)
input_c2, context_cnn2 = build_cnn_branch(input_dim_mms)
input_u2, utterance_cnn2 = build_cnn_branch(input_dim_mms)

# First fusion (context_cnn1 + utterance_cnn1) and (context_cnn2 + utterance_cnn2)
fused_1 = layers.Concatenate()([context_cnn1, utterance_cnn1])
fused_2 = layers.Concatenate()([context_cnn2, utterance_cnn2])



def build_fused_fcn(input_tensor):
    x = layers.Dense(512, activation="relu")(input_tensor)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    x = layers.Dense(8, activation="relu")(x)
    #x = layers.Dense(8, activation="relu")(x)
    return x

cnn1 = build_fused_fcn(fused_1)
cnn2 = build_fused_fcn(fused_2)

# Final fusion (cnn1 + cnn2)
final_fusion = layers.Concatenate()([cnn1, cnn2])

# Fully connected layers
fc = layers.Dense(64, activation="relu")(final_fusion)
fc = layers.Dense(32, activation="relu")(fc)
fc = layers.Dense(8, activation="relu")(fc)
output = layers.Dense(1, activation="sigmoid")(fc)

# Define and compile model
model = keras.Model(inputs=[input_c1, input_u1, input_c2, input_u2], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Model checkpoint
checkpoint_callback = ModelCheckpoint(
    "/kaggle/working/whisper+xlsr_cnn.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Train Model
model.fit(
    [Xc1_train, Xu1_train, Xc2_train, Xu2_train], y_train,
    epochs=100, batch_size=32,
    validation_data=([Xc1_val, Xu1_val, Xc2_val, Xu2_val], y_val),
    callbacks=[checkpoint_callback]
)

# Load best weights
model.load_weights("/kaggle/working/whisper+xlsr_cnn.weights.h5")
print("Loaded Best Model Weights.")

# Predictions
y_train_pred = (model.predict([Xc1_train, Xu1_train, Xc2_train, Xu2_train]) > 0.5).astype(int)
y_val_pred = (model.predict([Xc1_val, Xu1_val, Xc2_val, Xu2_val]) > 0.5).astype(int)
y_test_pred = (model.predict([Xc1_test, Xu1_test, Xc2_test, Xu2_test]) > 0.5).astype(int)

# Classification Reports
print("Train Set Classification Report:\n", classification_report(y_train, y_train_pred, digits=4))
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12            │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_13            │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_14            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_15            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_12 (Reshape)      │ (None, 512, 1)         │              0 │ input_layer_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_13 (Reshape)      │ (None, 512, 1)         │              0 │ input_layer_13[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_14 (Reshape)      │ (None, 1280, 1)        │              0 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_15 (Reshape)      │ (None, 1280, 1)        │              0 │ input_layer_15[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_24 (Conv1D)        │ (None, 510, 256)       │          1,024 │ reshape_12[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_26 (Conv1D)        │ (None, 510, 256)       │          1,024 │ reshape_13[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_28 (Conv1D)        │ (None, 1278, 256)      │          1,024 │ reshape_14[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_30 (Conv1D)        │ (None, 1278, 256)      │          1,024 │ reshape_15[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_24          │ (None, 255, 256)       │              0 │ conv1d_24[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_26          │ (None, 255, 256)       │              0 │ conv1d_26[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_28          │ (None, 639, 256)       │              0 │ conv1d_28[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_30          │ (None, 639, 256)       │              0 │ conv1d_30[0][0]        │
│ (MaxPooling1D)       

 Total params: 29,653,057 (113.12 MB)

 Trainable params: 29,653,057 (113.12 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.4499 - loss: 0.6937
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to /kaggle/working/whisper+xlsr_cnn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 211ms/step - accuracy: 0.4507 - loss: 0.6937 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/100
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5110 - loss: 0.6931
Epoch 2: val_accuracy did not improve from 0.50000
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5104 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/100
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5098 - loss: 0.6931
Epoch 3: val_accuracy did not improve from 0.50000
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5089 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/100
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4575 - loss: 0.6932
Epoch 4: val_accuracy did not improve from 0.50000
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

**FCN**

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CSV Files
df1 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_whisper.csv")  # LB dataset
df2 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_xlsr.csv")  # MMS dataset

# Extract Labels
y = df1["Sarcasm"].values  

# Extract features from both datasets
Xc1 = df1[[col for col in df1.columns if col.startswith("audio_c_feature_")]].values
Xu1 = df1[[col for col in df1.columns if col.startswith("audio_u_feature_")]].values
Xc2 = df2[[col for col in df2.columns if col.startswith("audio_c_feature_")]].values
Xu2 = df2[[col for col in df2.columns if col.startswith("audio_u_feature_")]].values

# Convert to NumPy arrays
Xc1, Xu1, Xc2, Xu2 = map(lambda x: np.array(x, dtype=np.float32), [Xc1, Xu1, Xc2, Xu2])
y = np.array(y, dtype=np.float32)

# Train-test split
Xc1_train, Xc1_temp, Xu1_train, Xu1_temp, Xc2_train, Xc2_temp, Xu2_train, Xu2_temp, y_train, y_temp = train_test_split(
    Xc1, Xu1, Xc2, Xu2, y, test_size=0.3, random_state=42, stratify=y
)

Xc1_val, Xc1_test, Xu1_val, Xu1_test, Xc2_val, Xc2_test, Xu2_val, Xu2_test, y_val, y_test = train_test_split(
    Xc1_temp, Xu1_temp, Xc2_temp, Xu2_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

# Define feature dimensions
input_dim_lb = 512  # LB Feature dimension
input_dim_mms = 1280  # MMS Feature dimension

# Fully connected network branch
def build_fcn_branch(input_dim):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Dense(256, activation="swish")(inp)
    x = layers.Dense(64, activation="swish")(x)
    x = layers.Dense(32, activation="swish")(x)
    x = layers.Dense(8, activation="swish")(x)
    # x = layers.Dense(64, activation="relu")(x)
    # x = layers.Dense(32, activation="relu")(x)
    return inp, x

# FCN for LB dataset
input_c1, context_fcn1 = build_fcn_branch(input_dim_lb)
input_u1, utterance_fcn1 = build_fcn_branch(input_dim_lb)

# FCN for MMS dataset
input_c2, context_fcn2 = build_fcn_branch(input_dim_mms)
input_u2, utterance_fcn2 = build_fcn_branch(input_dim_mms)

# First fusion
fused_1 = layers.Concatenate()([context_fcn1, utterance_fcn1])
fused_2 = layers.Concatenate()([context_fcn2, utterance_fcn2])

# Additional FCN model after fusion
def build_fcn_model(input_tensor):
    x = layers.Dense(128, activation="swish")(input_tensor)
    x = layers.Dense(64, activation="swish")(x)
    x = layers.Dense(32, activation="swish")(x)
    x = layers.Dense(8, activation="swish")(x)
   # x = layers.Dense(1, activation="relu")(x)
    return x

fcn1 = build_fcn_model(fused_1)
fcn2 = build_fcn_model(fused_2)

# Final fusion
final_fusion = layers.Concatenate()([fcn1, fcn2])

# Fully connected layers
fc = layers.Dense(64, activation="swish")(final_fusion)
fc = layers.Dense(32, activation="swish")(fc)
fc = layers.Dense(8, activation="swish")(fc)
output = layers.Dense(1, activation="sigmoid")(fc)

# Define and compile model
model = keras.Model(inputs=[input_c1, input_u1, input_c2, input_u2], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Model checkpoint
checkpoint_callback = ModelCheckpoint(
    "/kaggle/working/whisper+xlsr_fcn.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Train Model
model.fit(
    [Xc1_train, Xu1_train, Xc2_train, Xu2_train], y_train,
    epochs=50, batch_size=32,
    validation_data=([Xc1_val, Xu1_val, Xc2_val, Xu2_val], y_val),
    callbacks=[checkpoint_callback]
)

# Load best weights
model.load_weights("/kaggle/working/whisper+xlsr_fcn.weights.h5")
print("Loaded Best Model Weights.")

# Predictions
y_train_pred = (model.predict([Xc1_train, Xu1_train, Xc2_train, Xu2_train]) > 0.5).astype(int)
y_val_pred = (model.predict([Xc1_val, Xu1_val, Xc2_val, Xu2_val]) > 0.5).astype(int)
y_test_pred = (model.predict([Xc1_test, Xu1_test, Xc2_test, Xu2_test]) > 0.5).astype(int)

# Classification Reports
print("Train Set Classification Report:\n", classification_report(y_train, y_train_pred, digits=4))
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_66            │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_67            │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_68            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_69            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_456 (Dense)         │ (None, 256)            │        131,328 │ input_layer_66[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_460 (Dense)         │ (None, 256)            │        131,328 │ input_layer_67[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_464 (Dense)         │ (None, 256)            │        327,936 │ input_layer_68[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_468 (Dense)         │ (None, 256)            │        327,936 │ input_layer_69[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_457 (Dense)         │ (None, 64)             │         16,448 │ dense_456[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_461 (Dense)         │ (None, 64)             │         16,448 │ dense_460[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_465 (Dense)         │ (None, 64)             │         16,448 │ dense_464[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_469 (Dense)         │ (None, 64)             │         16,448 │ dense_468[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_458 (Dense)         │ (None, 32)             │          2,080 │ dense_457[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_462 (Dense)         │ (None, 32)             │          2,080 │ dense_461[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_466 (Dense)         │ (None, 32)             │          2,080 │ dense_465[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_470 (Dense)         │ (None, 32)             │          2,080 │ dense_469[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_459 (Dense)         │ (None, 8)              │            264 │ dense_458[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_463 (Dense)    

 Total params: 1,022,689 (3.90 MB)

 Trainable params: 1,022,689 (3.90 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.4868 - loss: 0.6934
Epoch 1: val_accuracy improved from -inf to 0.46667, saving model to /kaggle/working/whisper+xlsr_fcn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 161ms/step - accuracy: 0.4875 - loss: 0.6934 - val_accuracy: 0.4667 - val_loss: 0.6826
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5184 - loss: 0.6941
Epoch 2: val_accuracy improved from 0.46667 to 0.62500, saving model to /kaggle/working/whisper+xlsr_fcn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5200 - loss: 0.6938 - val_accuracy: 0.6250 - val_loss: 0.6670
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5594 - loss: 0.6697
Epoch 3: val_accuracy did not improve from 0.62500
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5593 - loss: 0.6699 - val_accuracy: 0.6000 - val_loss: 0.6683
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6055 - loss: 0.6555
Epoch 4: val_accuracy did not impr

# MMS + XLS-R
**CNN**

In [46]:
"""
each branch gets a separate cnn
"""

from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CSV Files
df1 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_mms.csv")  # LB dataset
df2 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_xlsr.csv")  # MMS dataset

# Extract Labels (assuming both datasets have the same labels)
y = df1["Sarcasm"].values  

# Extract features from both datasets
Xc1 = df1[[col for col in df1.columns if col.startswith("audio_c_feature_")]].values
Xu1 = df1[[col for col in df1.columns if col.startswith("audio_u_feature_")]].values
Xc2 = df2[[col for col in df2.columns if col.startswith("audio_c_feature_")]].values
Xu2 = df2[[col for col in df2.columns if col.startswith("audio_u_feature_")]].values

# Convert to NumPy arrays
Xc1, Xu1, Xc2, Xu2 = map(lambda x: np.array(x, dtype=np.float32), [Xc1, Xu1, Xc2, Xu2])
y = np.array(y, dtype=np.float32)

# Train-test split (70%-30%)
Xc1_train, Xc1_temp, Xu1_train, Xu1_temp, Xc2_train, Xc2_temp, Xu2_train, Xu2_temp, y_train, y_temp = train_test_split(
    Xc1, Xu1, Xc2, Xu2, y, test_size=0.3, random_state=42, stratify=y
)

# Further split temp set into validation (10%) and test (20%)
Xc1_val, Xc1_test, Xu1_val, Xu1_test, Xc2_val, Xc2_test, Xu2_val, Xu2_test, y_val, y_test = train_test_split(
    Xc1_temp, Xu1_temp, Xc2_temp, Xu2_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

# Define feature dimensions separately
input_dim_lb = 1280  # LB Feature dimension
input_dim_mms = 1280  # MMS Feature dimension

# CNN Model for feature extraction
def build_cnn_branch(input_dim):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Reshape((input_dim, 1))(inp)
    x = layers.Conv1D(filters=126, kernel_size=3, activation="swish")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=64, kernel_size=3, activation="swish")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    return inp, x

# Apply CNN on LB inputs (768)
input_c1, context_cnn1 = build_cnn_branch(input_dim_lb)
input_u1, utterance_cnn1 = build_cnn_branch(input_dim_lb)

# Apply CNN on MMS inputs (1280)
input_c2, context_cnn2 = build_cnn_branch(input_dim_mms)
input_u2, utterance_cnn2 = build_cnn_branch(input_dim_mms)

# First fusion (context_cnn1 + utterance_cnn1) and (context_cnn2 + utterance_cnn2)
fused_1 = layers.Concatenate()([context_cnn1, utterance_cnn1])
fused_2 = layers.Concatenate()([context_cnn2, utterance_cnn2])



def build_fused_fcn(input_tensor):
    x = layers.Dense(128, activation="swish")(input_tensor)
    x = layers.Dense(64, activation="swish")(x)
    x = layers.Dense(32, activation="swish")(x)
    return x

cnn1 = build_fused_fcn(fused_1)
cnn2 = build_fused_fcn(fused_2)

# Final fusion (cnn1 + cnn2)
final_fusion = layers.Concatenate()([cnn1, cnn2])

# Fully connected layers
fc = layers.Dense(64, activation="swish")(final_fusion)
fc = layers.Dense(32, activation="swish")(fc)
output = layers.Dense(1, activation="sigmoid")(fc)

# Define and compile model
model = keras.Model(inputs=[input_c1, input_u1, input_c2, input_u2], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Model checkpoint
checkpoint_callback = ModelCheckpoint(
    "/kaggle/working/mms+xlsr_cnn.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Train Model
model.fit(
    [Xc1_train, Xu1_train, Xc2_train, Xu2_train], y_train,
    epochs=50, batch_size=32,
    validation_data=([Xc1_val, Xu1_val, Xc2_val, Xu2_val], y_val),
    callbacks=[checkpoint_callback]
)

# Load best weights
model.load_weights("/kaggle/working/mms+xlsr_cnn.weights.h5")
print("Loaded Best Model Weights.")

# Predictions
y_train_pred = (model.predict([Xc1_train, Xu1_train, Xc2_train, Xu2_train]) > 0.5).astype(int)
y_val_pred = (model.predict([Xc1_val, Xu1_val, Xc2_val, Xu2_val]) > 0.5).astype(int)
y_test_pred = (model.predict([Xc1_test, Xu1_test, Xc2_test, Xu2_test]) > 0.5).astype(int)

# Classification Reports
print("Train Set Classification Report:\n", classification_report(y_train, y_train_pred, digits=4))
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


Model: "functional_45"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_180           │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_181           │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_182           │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_183           │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_120 (Reshape)     │ (None, 1280, 1)        │              0 │ input_layer_180[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_121 (Reshape)     │ (None, 1280, 1)        │              0 │ input_layer_181[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_122 (Reshape)     │ (None, 1280, 1)        │              0 │ input_layer_182[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_123 (Reshape)     │ (None, 1280, 1)        │              0 │ input_layer_183[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_336 (Conv1D)       │ (None, 1278, 126)      │            504 │ reshape_120[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_338 (Conv1D)       │ (None, 1278, 126)      │            504 │ reshape_121[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_340 (Conv1D)       │ (None, 1278, 126)      │            504 │ reshape_122[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_342 (Conv1D)       │ (None, 1278, 126)      │            504 │ reshape_123[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_336         │ (None, 639, 126)       │              0 │ conv1d_336[0][0]       │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_338         │ (None, 639, 126)       │              0 │ conv1d_338[0][0]       │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_340         │ (None, 639, 126)       │              0 │ conv1d_340[0][0]       │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_342         │ (None, 639, 126)       │              0 │ conv1d_342[0][0]       │
│ (MaxPooling1D)       

 Total params: 10,546,465 (40.23 MB)

 Trainable params: 10,546,465 (40.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.5515 - loss: 0.6912
Epoch 1: val_accuracy improved from -inf to 0.51667, saving model to /kaggle/working/mms+xlsr_cnn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 129ms/step - accuracy: 0.5515 - loss: 0.6910 - val_accuracy: 0.5167 - val_loss: 0.6872
Epoch 2/50
26/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5650 - loss: 0.6791
Epoch 2: val_accuracy improved from 0.51667 to 0.64167, saving model to /kaggle/working/mms+xlsr_cnn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5661 - loss: 0.6789 - val_accuracy: 0.6417 - val_loss: 0.6612
Epoch 3/50
26/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5948 - loss: 0.6567
Epoch 3: val_accuracy did not improve from 0.64167
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5943 - loss: 0.6567 - val_accuracy: 0.6250 - val_loss: 0.6578
Epoch 4/50
26/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6146 - loss: 0.6480
Epoch 4: val_accuracy improved from 0.6

**FCN**

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CSV Files
df1 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_mms.csv")  # LB dataset
df2 = pd.read_csv("/kaggle/input/btp-audioembeddings/audio_features_xlsr.csv")  # MMS dataset

# Extract Labels
y = df1["Sarcasm"].values  

# Extract features from both datasets
Xc1 = df1[[col for col in df1.columns if col.startswith("audio_c_feature_")]].values
Xu1 = df1[[col for col in df1.columns if col.startswith("audio_u_feature_")]].values
Xc2 = df2[[col for col in df2.columns if col.startswith("audio_c_feature_")]].values
Xu2 = df2[[col for col in df2.columns if col.startswith("audio_u_feature_")]].values

# Convert to NumPy arrays
Xc1, Xu1, Xc2, Xu2 = map(lambda x: np.array(x, dtype=np.float32), [Xc1, Xu1, Xc2, Xu2])
y = np.array(y, dtype=np.float32)

# Train-test split
Xc1_train, Xc1_temp, Xu1_train, Xu1_temp, Xc2_train, Xc2_temp, Xu2_train, Xu2_temp, y_train, y_temp = train_test_split(
    Xc1, Xu1, Xc2, Xu2, y, test_size=0.3, random_state=42, stratify=y
)

Xc1_val, Xc1_test, Xu1_val, Xu1_test, Xc2_val, Xc2_test, Xu2_val, Xu2_test, y_val, y_test = train_test_split(
    Xc1_temp, Xu1_temp, Xc2_temp, Xu2_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

# Define feature dimensions
input_dim_lb = 1280  # LB Feature dimension
input_dim_mms = 1280  # MMS Feature dimension

# Fully connected network branch
def build_fcn_branch(input_dim):
    inp = keras.Input(shape=(input_dim,))
    x = layers.Dense(512, activation="swish")(inp)
    x = layers.Dense(256, activation="swish")(x)
    x = layers.Dense(128, activation="swish")(x)
    x = layers.Dense(64, activation="swish")(x)
    x = layers.Dense(32, activation="swish")(x)
    return inp, x

# FCN for LB dataset
input_c1, context_fcn1 = build_fcn_branch(input_dim_lb)
input_u1, utterance_fcn1 = build_fcn_branch(input_dim_lb)

# FCN for MMS dataset
input_c2, context_fcn2 = build_fcn_branch(input_dim_mms)
input_u2, utterance_fcn2 = build_fcn_branch(input_dim_mms)

# First fusion
fused_1 = layers.Concatenate()([context_fcn1, utterance_fcn1])
fused_2 = layers.Concatenate()([context_fcn2, utterance_fcn2])

# Additional FCN model after fusion
def build_fcn_model(input_tensor):
    x = layers.Dense(512, activation="swish")(input_tensor)
    x = layers.Dense(256, activation="swish")(x)
    x = layers.Dense(128, activation="swish")(x)
    return x

fcn1 = build_fcn_model(fused_1)
fcn2 = build_fcn_model(fused_2)

# Final fusion
final_fusion = layers.Concatenate()([fcn1, fcn2])

# Fully connected layers
fc = layers.Dense(64, activation="swish")(final_fusion)
fc = layers.Dense(32, activation="swish")(fc)
fc = layers.Dense(8, activation="swish")(fc)
output = layers.Dense(1, activation="sigmoid")(fc)

# Define and compile model
model = keras.Model(inputs=[input_c1, input_u1, input_c2, input_u2], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Model checkpoint
checkpoint_callback = ModelCheckpoint(
    "/kaggle/working/mms+xlsr_fcn.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

# Train Model
model.fit(
    [Xc1_train, Xu1_train, Xc2_train, Xu2_train], y_train,
    epochs=50, batch_size=32,
    validation_data=([Xc1_val, Xu1_val, Xc2_val, Xu2_val], y_val),
    callbacks=[checkpoint_callback]
)

# Load best weights
model.load_weights("/kaggle/working/mms+xlsr_fcn.weights.h5")
print("Loaded Best Model Weights.")

# Predictions
y_train_pred = (model.predict([Xc1_train, Xu1_train, Xc2_train, Xu2_train]) > 0.5).astype(int)
y_val_pred = (model.predict([Xc1_val, Xu1_val, Xc2_val, Xu2_val]) > 0.5).astype(int)
y_test_pred = (model.predict([Xc1_test, Xu1_test, Xc2_test, Xu2_test]) > 0.5).astype(int)

# Classification Reports
print("Train Set Classification Report:\n", classification_report(y_train, y_train_pred, digits=4))
print("Validation Set Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
print("Test Set Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_37            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_38            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_39            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_40            │ (None, 1280)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_251 (Dense)         │ (None, 512)            │        655,872 │ input_layer_37[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_256 (Dense)         │ (None, 512)            │        655,872 │ input_layer_38[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_261 (Dense)         │ (None, 512)            │        655,872 │ input_layer_39[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_266 (Dense)         │ (None, 512)            │        655,872 │ input_layer_40[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_252 (Dense)         │ (None, 256)            │        131,328 │ dense_251[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_257 (Dense)         │ (None, 256)            │        131,328 │ dense_256[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_262 (Dense)         │ (None, 256)            │        131,328 │ dense_261[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_267 (Dense)         │ (None, 256)            │        131,328 │ dense_266[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_253 (Dense)         │ (None, 128)            │         32,896 │ dense_252[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_258 (Dense)         │ (None, 128)            │         32,896 │ dense_257[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_263 (Dense)         │ (None, 128)            │         32,896 │ dense_262[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_268 (Dense)         │ (None, 128)            │         32,896 │ dense_267[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_254 (Dense)         │ (None, 64)             │          8,256 │ dense_253[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_259 (Dense)    

 Total params: 3,735,537 (14.25 MB)

 Trainable params: 3,735,537 (14.25 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.5209 - loss: 0.6946
Epoch 1: val_accuracy improved from -inf to 0.60000, saving model to /kaggle/working/mms+xlsr_fcn.weights.h5
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 126ms/step - accuracy: 0.5207 - loss: 0.6945 - val_accuracy: 0.6000 - val_loss: 0.6840
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5452 - loss: 0.6813
Epoch 2: val_accuracy did not improve from 0.60000
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5458 - loss: 0.6812 - val_accuracy: 0.5833 - val_loss: 0.6615
Epoch 3/50
15/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5784 - loss: 0.6642 
Epoch 3: val_accuracy did not improve from 0.60000
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5836 - loss: 0.6631 - val_accuracy: 0.5917 - val_loss: 0.6764
Epoch 4/50
14/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6242 - loss: 0.6584 
Epoch 4: val_accuracy did not improve from 0.60000
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 